# Profiling APIs


In [11]:
import os
import sys

# Assume the current working directory is where the this notebook file is located 
project_dir = os.path.dirname(os.getcwd())
if project_dir not in sys.path:
    sys.path.append(os.path.join(project_dir, "src"))
# Alternatively you can install package directly from the Pypi. 
# %pip install im2im

In [12]:
%pip install py-cpuinfo
%pip install gputil
import cpuinfo
import GPUtil
cpu_name = cpuinfo.get_cpu_info()['brand_raw']
print(f"CPU: {cpu_name}")

gpus = GPUtil.getGPUs()
print(f"GPU:0: {GPUtil.getGPUs()[0].name}")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
CPU: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz
GPU:0: NVIDIA GeForce GTX 1080 Ti


In [13]:
import cProfile
import pstats

from im2im import im2im_code, _code_generator

def clear_cache_in_code_generator():
    _code_generator._cache = {}
    
clear_cache_in_code_generator()
with cProfile.Profile() as profile:
    code = im2im_code("source", {"lib": "numpy"}, "target", {"lib": "torch", "color_channel":"gray", "image_dtype": "uint8"})

    results = pstats.Stats(profile)
    results.print_stats(0)

    print(f'Conversion code from "source" to "target" is \n{code}')

         1684 function calls in 0.001 seconds

Converson code from "source" to "target" is 
import torch
from torchvision.transforms import functional as v1F
image = torch.from_numpy(source)
image = image.permute(2, 0, 1)
image = image.unsqueeze(0)
target = v1F.rgb_to_grayscale(image)


In [15]:
import numpy as np
from im2im import im2im

clear_cache_in_code_generator()
source = np.random.randint(0, 256, (2000, 2000, 3), dtype=np.uint8)
with cProfile.Profile() as profile:
    target = im2im(source, {"lib": "numpy"}, {"lib": "torch", "color_channel":"gray", "image_dtype": "uint8"})

    results = pstats.Stats(profile)
    results.print_stats(0)

    print(type(target), target.shape, target.dtype)

         1712 function calls in 0.023 seconds

<class 'torch.Tensor'> torch.Size([1, 1, 2000, 2000]) torch.uint8


In [16]:
from im2im import im2im_path

clear_cache_in_code_generator()
with cProfile.Profile() as profile:
    steps = im2im_path({"lib": "numpy"}, {"lib": "torch", "color_channel":"gray", "image_dtype": "uint8"})
    
    results = pstats.Stats(profile)
    results.print_stats(0)

    for step in steps:
        print(step)

         1510 function calls in 0.001 seconds

{'data_representation': 'numpy.ndarray', 'color_channel': 'rgb', 'channel_order': 'channel last', 'minibatch_input': False, 'image_data_type': 'uint8', 'device': 'cpu'}
{'data_representation': 'torch.tensor', 'color_channel': 'rgb', 'channel_order': 'channel last', 'minibatch_input': False, 'image_data_type': 'uint8', 'device': 'cpu'}
{'data_representation': 'torch.tensor', 'color_channel': 'rgb', 'channel_order': 'channel first', 'minibatch_input': False, 'image_data_type': 'uint8', 'device': 'cpu'}
{'data_representation': 'torch.tensor', 'color_channel': 'rgb', 'channel_order': 'channel first', 'minibatch_input': True, 'image_data_type': 'uint8', 'device': 'cpu'}
{'data_representation': 'torch.tensor', 'color_channel': 'gray', 'channel_order': 'channel first', 'minibatch_input': True, 'image_data_type': 'uint8', 'device': 'cpu'}
